In [15]:
import sys # for automation and parallelization: set manual to false when run by a launcher
import json
 
default = {'scenario': 'houston','training_folder':'../..'} # Default execution parameters
manual, argv = (True, default) if 'ipykernel' in sys.argv[0] else (False, dict(default, **json.loads(sys.argv[1])))


In [16]:
import os
import time
import geopandas as gpd
import pandas as pd
sys.path.insert(0, r'../../../quetzal') # Add path to quetzal
import numpy as np
from typing import Literal

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
io_engine = 'pyogrio' if on_lambda else 'pyogrio' #or fiona

In [17]:

scenario = argv['scenario']

on_lambda = bool(os.environ.get('AWS_EXECUTION_ENV'))
print('On Lambda : ', on_lambda)

training_folder = argv['training_folder']
input_folder = training_folder +r'/inputs/'

if not on_lambda:
    scenario_folder = training_folder + '/scenarios/' + scenario + '/inputs/'
    output_folder = training_folder + '/scenarios/' + scenario + '/outputs/'
    model_folder = training_folder + '/scenarios/' + scenario + '/model/'
else:
    scenario_folder = input_folder
    output_folder = training_folder + '/outputs/'
    model_folder = training_folder + '/model/'
print('input folder: ', input_folder)
print('output folder: ', output_folder)
print('scen folder : ', scenario_folder)
print('model folder : ', model_folder)

On Lambda :  False
input folder:  ../../inputs/
output folder:  ../../scenarios/houston/outputs/
scen folder :  ../../scenarios/houston/inputs/
model folder :  ../../scenarios/houston/model/


In [18]:
training_folder

'../..'

# format zones

In [19]:
zones = gpd.read_file(input_folder + 'zones/{}.geojson'.format(scenario), engine=io_engine) .to_crs(4326)
zones = zones.set_index('OBJECTID')
zones.index = 'zone_bc_' + zones.index.astype(str)
zones.index.name = 'index'

tmp_zones = zones[['pop_dens', 'pop_total','emp_total', 'geometry', 'NAME']].rename(columns={'pop_dens': 'density','pop_total':'population', 'emp_total': 'jobs' })

In [20]:
output_folder = training_folder+'/scenarios/{scen}/inputs/'.format(scen = scenario)
try:
            os.makedirs(output_folder)
except FileExistsError:
            pass

tmp_zones.to_file(output_folder + 'zones.geojson',driver='GeoJSON')

In [21]:
# Peut être verification à faire

# find meters CRS
from shapely.geometry import Point, LineString
# from quetzal_cyclops
def get_epsg(lat: float, lon: float) -> int:
    '''
    lat, lon or y, x
    return EPSG in meter for a given (lat,lon)
    lat is north south 
    lon is est west
    '''
    return int(32700 - round((45 + lat) / 90, 0) * 100 + round((183 + lon) / 6, 0))

centroid = [*LineString(zones.centroid.values).centroid.coords][0]
crs = get_epsg(centroid[1],centroid[0])
crs

zones['area (km2)'] = zones.to_crs(crs).area
zones['area (km2)'].sum()